In [10]:
import requests
import numpy as np
from tqdm import tqdm

url = 'https://rest.uniprot.org/uniprotkb/stream?compressed=true&download=true&format=fasta&query=%28%28taxonomy_id%3A9606%29+OR+%28taxonomy_id%3A4930%29+OR+%28taxonomy_id%3A10088%29+OR+%28taxonomy_id%3A3702%29+OR+%28taxonomy_id%3A9544%29+OR+%28taxonomy_id%3A10114%29+OR+%28taxonomy_id%3A7955%29+OR+%28taxonomy_id%3A39947%29+OR+%28taxonomy_id%3A562%29+OR+%28taxonomy_id%3A2157%29+OR+%28taxonomy_id%3A7215%29+OR+%28taxonomy_id%3A4564%29%29'
destination = 'data/uniprot_dataset.tsv.gz'


In [11]:


def download_file_with_progress(url, destination):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte

    with open(destination, 'wb') as file, tqdm(
        desc=destination,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(block_size):
            bar.update(len(data))
            file.write(data)

# Example usage:
download_file_with_progress(url, destination)

data/uniprot_dataset.tsv.gz: 9.34MiB [25:55, 6.30kiB/s]


ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [29]:
import gzip
fa = gzip.open('data/GRCh38_latest_rna.fna.gz', 'r')
data = fa.read().decode()
records = data.split('>')

In [52]:
processed = []
for record in records:
    if not record:
        continue
    ann, seq = record.split('\n', 1)
    seq = seq.replace('\n', '')
    processed.append({'annotation': ann, 'sequence': seq})

In [46]:
list(processed.items())[0]

('NM_000014.6 Homo sapiens alpha-2-macroglobulin (A2M), transcript variant 1, mRNA',
 'GGGACCAGATGGATTGTAGGGAGTAGGGTACAATACAGTCTGTTCTCCTCCAGCTCCTTCTTTCTGCAACATGGGGAAGAACAAACTCCTTCATCCAAGTCTGGTTCTTCTCCTCTTGGTCCTCCTGCCCACAGACGCCTCAGTCTCTGGAAAACCGCAGTATATGGTTCTGGTCCCCTCCCTGCTCCACACTGAGACCACTGAGAAGGGCTGTGTCCTTCTGAGCTACCTGAATGAGACAGTGACTGTAAGTGCTTCCTTGGAGTCTGTCAGGGGAAACAGGAGCCTCTTCACTGACCTGGAGGCGGAGAATGACGTACTCCACTGTGTCGCCTTCGCTGTCCCAAAGTCTTCATCCAATGAGGAGGTAATGTTCCTCACTGTCCAAGTGAAAGGACCAACCCAAGAATTTAAGAAGCGGACCACAGTGATGGTTAAGAACGAGGACAGTCTGGTCTTTGTCCAGACAGACAAATCAATCTACAAACCAGGGCAGACAGTGAAATTTCGTGTTGTCTCCATGGATGAAAACTTTCACCCCCTGAATGAGTTGATTCCACTAGTATACATTCAGGATCCCAAAGGAAATCGCATCGCACAATGGCAGAGTTTCCAGTTAGAGGGTGGCCTCAAGCAATTTTCTTTTCCCCTCTCATCAGAGCCCTTCCAGGGCTCCTACAAGGTGGTGGTACAGAAGAAATCAGGTGGAAGGACAGAGCACCCTTTCACCGTGGAGGAATTTGTTCTTCCCAAGTTTGAAGTACAAGTAACAGTGCCAAAGATAATCACCATCTTGGAAGAAGAGATGAATGTATCAGTGTGTGGCCTATACACATATGGGAAGCCTGTCCCTGGACATGTGACTGTGAGCATTTGCAGAAAGTATAGTGACGCTTCCGACTGCCACGGTGAA

In [72]:
df = pd.DataFrame.from_dict(data=processed)
df.annotation.str.contains('uncharacterized').sum() / df.shape[0]

0.12055898574445903

In [73]:
df.annotation = df.annotation.apply(lambda s: ' '.join(s.split(' ')[1:]).strip())

In [77]:
print(df.shape)
df.head()


(185121, 2)


,annotation,sequence
0,"Homo sapiens alpha-2-macroglobulin (A2M), tran...",GGGACCAGATGGATTGTAGGGAGTAGGGTACAATACAGTCTGTTCT...
1,"Homo sapiens N-acetyltransferase 2 (NAT2), mRNA",ACTTTATTACAGACCTTGGAAGCAAGAGGATTGCATTCAGCCTAGT...
2,Homo sapiens acyl-CoA dehydrogenase medium cha...,AGAGGAGTCCCGCGTTCGGGGAGTATGTCAAGGCCGTGACCCGTGT...
3,Homo sapiens acyl-CoA dehydrogenase short chai...,ACTCCGGAACAGCGCGCTCGCAGCGGGAGGTCGCGAAGCCTGGGAC...
4,Homo sapiens acyl-CoA dehydrogenase very long ...,AGAGCTGGGTCAGAGCTCGAGCCAGCGGCGCCCGGAGAGATTCGGA...


In [75]:
df.to_csv('data/sequences_annotations.tsv', index=False)